In [1]:
import os
import importlib

import config
import gpt
import AbstractModel

importlib.reload(config)
importlib.reload(gpt)
importlib.reload(AbstractModel)
config.source_dot_env()


from openai import OpenAI
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# model = "gpt-3.5-turbo"
# prompt = "haiku in dutch about deep learning"
# max_tokens = 500


# completion = client.chat.completions.create(
#     model=model,
#     messages=[{"role": "user", "content": prompt}],
#     max_tokens=max_tokens,
# )

In [2]:
# model = gpt.GPTModel()
# outputs, meta = model(["haiku in dutch about deep learning", "short poem in spanish about cheese"])
# print(outputs)
# print(f"price = ${model.compute_price(meta)}")

['Diep leren ontbrandt\nNeuronen vuren snel\nWijsheid groeit gestaag', 'Queso amado, dulce manjar,\nen tu sabor encuentro paz,\nqueso fresco, queso curado,\nen cada bocado soy inundado.\n\nCon pan o vino te disfruto,\nen una tabla o en un tazón,\nqueso, eres mi más fiel tributo,\nmi delicia, mi pasión.\n\nQueso, joya de la gastronomía,\nte venero con devoción,\nporque contigo, mi alegría\nes pura y completa emoción.']
price = $0.0002225


In [16]:
import random
PROMPT_SYTHNETIC_SMART = """
You're a student taking a course where you're tasked with writing a SMART goal defining your perosonal learning objective for the course.
A SMART learning goal must be Specific, Measurable, Action-oriented, Relevant, and Time-bound as described further below:

Specific: Concrete and unambiguous definition of the goal: is it clear what you want to achieve?

Measurable: It is clear what information you use to determine whether the learning objective has been achieved. This can be named both quantitatively and qualitatively; for example, "X positive comments about "Y" from fellow students".

Action-oriented: It is clear what behaviour you exhibit when you have achieved your goal.

Relevant: The learning objective is based on your own analysis and fits within the assessment criteria.

Time-bound: The learning objective specifically indicates when the goal is achieved and is realistic for a 4-week project.


Furthermore, an "action plan" for a SMART goal should:
* contain concrete actions you will carry out to achieve your goal.
* Include how you will collect information on your progress in the meantime, so that you know whether you are on the right track and can make any necessary adjustments in the meantime.
* Be clear and follow logically from the formulated learning objective  .
* Be achievable within the stipulated time.
* Allow for interim evaluation and adjustment.


Your response should contain a SMART goal written from your perspective as a student, as well as an appropriate action plan, formatted like the example below (all as plain text with no markdown formatting).

Example response follows:
Learning objective: "My goal is to make eye contact with the audience during my video pitch and literature presentation to better engage the viewer with my story. In the video pitch, I do this by looking closely into the camera most of the time and in the literature presentation by alternately looking at the different people in the audience, both my fellow students and teachers."
Action plan: "I will achieve this by not memorising my story verbatim, but rather by using keywords and by practising looking into the camera and making video recordings of this during the preparation of my pitch. I also practise my literature presentation dry where my group mates sit on opposite sides of the room. In the video recordings and feedback from my group mates afterwards, I will check whether I indeed make good eye contact"

If we analyse this learning objective as an example with the aforementioned criteria: we see that it is indeed SMART in its formulation:
Specifics: Making eye contact with the audience
Measurable: It is clear how this can be measured, as you can see in a video recording or ask the audience if eye contact is indeed made -> family or friends you can also ask for feedback!
Action-oriented: It is clear exactly what you are doing when the goal is achieved: "looking well into the camera most of the time", and "alternating between looking at different people in the audience".
Relevant: "Making eye contact" fits within the assessment criterion "non-verbal communication" and can be found on the inspiration list for key sub-skills in presentation [see assessment criteria].
Time-bound: "during my video pitch and literature presentation"

And Analyzing the sample action plan we see it specifically indicates which steps are taken in preparation ("don't literally memorise", "make video recordings and check if people are looking into the camera", and "by asking feedback from group members after a live presentation"). The video recordings and feedback provide the right information on progress, allowing you to make adjustments.

In your case the goal you're writing should focus on '{skill}' (for example {skill_description}).
Now respond only with a SMART goal written from your perspective as a student as well as an appropriate action plan.
""".strip()

# List of sub-skills for presenting to inspire you in setting your personal learning goal:
SKILLS = {
    "Clarity in communication": "The ability to explain complex concepts and ideas in a simple and understandable way, but also to use effective signal and reference words to involve the audience in the story",

    "Structure of the presentation": "The ability to structure a presentation logically and coherently, with a clear introduction, middle and conclusion, and to make clear connections between different parts.",

    "Language use": "The ability to use precise, clear and professional language, including the appropriate use of scientific terminology. Spaces between sentences are not filled with distracting filler words, but rather used strategically to get the message across to the audience.",

    "Time management and speaking pace": "The ability to use an appropriate pace of speech that is comfortable for the audience to follow and that fits the story within time without missing important points.",

    "Pitch and intonation": "Using different pitches and intonation to make the presentation more engaging and understandable. Intonations can also be used to place emphasis on a specific part of a sentence to better guide the audience through the story.",

    "Voice volume": "The ability to use the voice in such a way that everyone can hear the presentation clearly.",

    "Non-verbal communication": "Eye contact: the ability to make contact with the audience and involve them in the story by consciously looking at them (e.g. looking into the camera) and distributing attention evenly across the audience",

    "Facial Expression: The ability to use facial expressions to give an accessible presentation (e.g., friendly smile) and to emphasize what is being said (e.g., by using facial expressions).":" ",

    # TODO: "[see also the knowledge clip about hand gestures and body language]":
    "Hand gestures": "The ability to use hand gestures that support and thereby reinforce the spoken message. Consider, for example, counting with the hands when 3 main points are mentioned, or emphasizing the 'one' versus the 'other' side of the matter, etc. !.",
}


model = gpt.GPTModel() # gpt 3
#model = gpt.GPTModel("gpt-4-0125-preview")
skills = []
prompts = []
for i in range(20):
    skill = random.choice(list(SKILLS.keys()))
    skill_description = SKILLS[skill]
    skills.append(skill)

    prompt = PROMPT_SYTHNETIC_SMART.format(skill=skill, skill_description=skill_description)
    #print("\n\n", prompt)
    prompts.append(prompt)

outputs, meta = model(prompts)

In [17]:
import csv

# Define the header for the CSV file
header = ['Skill', 'Output']

# Create a list of rows to write to the CSV file
rows = []
for output, skill in zip(outputs, skills):
    rows.append([skill, output])

# Write the rows to the CSV file
with open('synthetic_smart.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)


In [18]:
# gpt 3:
print(f"price = ${model.compute_price(meta)}")
print(outputs[0])

price = $0.0117775
Learning objective: "My goal is to enhance my language use by using precise, clear, and professional language, including the appropriate use of scientific terminology, in my written assignments and class discussions in order to effectively communicate complex ideas to my audience."

Action plan: I will achieve this by first identifying key scientific terminology relevant to the course material and incorporating it into my assignments and class discussions. I will also practice writing concise sentences and actively eliminating filler words to ensure the clarity and professionalism of my language use. In addition, I will seek feedback from my peers and instructors on my language use to track my progress and make necessary adjustments.


In [11]:
print(f"price = ${model.compute_price(meta)}")
print(outputs[0])

price = $0.02441
Learning Objective: My goal is to enhance my understanding and application of statistical analysis using Python, specifically by completing a project where I accurately apply at least three different statistical methods—such as regression analysis, ANOVA, and chi-squared tests—to a dataset by the end of this 4-week course. This will not only demonstrate my proficiency in Python for statistical applications but also improve my ability to interpret data outcomes effectively, contributing to my overall course performance.

Action Plan:
1. **Week 1:** 
   - **Identify and Gather Resources:** Research and compile a list of resources—books, online tutorials, and academic papers—that cover the use of Python for statistical analysis, with a focus on regression analysis, ANOVA, and chi-squared tests.
   - **Begin Learning:** Start with regression analysis by reading selected materials and practising coding examples. Create a log to track hours spent studying and coding, along w